In [ ]:
from profit_scanner.profit_scanner import ProfitScanner
from transaction.transaction_loader import TransactionLoader

In [ ]:
#enable autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
owner_address = "0x1a869357c99bdf201D0378971FfE0b78DA5697E0"

transaction_loader = TransactionLoader()
normal_transactions, erc20_transactions, internal_transactions = transaction_loader.load_transactions(owner_address)
profit_scanner = ProfitScanner(owner_address, normal_transactions, erc20_transactions, internal_transactions)

In [ ]:
res = profit_scanner.process()

In [ ]:
res

In [ ]:
data = [r.to_json() for r in res]

In [ ]:
data